### Vorgehen:
1. beschränktere Parameter herausfinden durch Experimente mit num_exp=1 und reduce_dataset=0.2.
    Parameter: Backbone, kd(schhonmal testen)
2. Wie 1. aber mit num_exp=3.
    Parameter: classification, selection_method
- Dadurch schonmal grobe Infos sammeln und dann später stichprobenartig verifizieren

3. Mit den gesammelten Infos die Benchmarks übertreffen oder Schonmal Q2 und Q3 anfangen

## ---

### Sweep: classification_and_selection
- reduce_dataset=1.1 statt 0.2:   
  
|        | cifar100 | imagenetr | cub | dil_imagenetr | imageneta | vtab | cars | omnibenchmark | limited_domainnet |
|--------|----------|-----------|-----|--------------|-----------|------|------|--------------|-------------------|
| Selection| bayesian | bayesian | bayesian | bayesian | bayesian | bayesian | --- | bayesian | bayesian |
| Classification  | inv_ws/inv_kl | ws | inv_eucld | ws | ws | kl/eucld | --- | ws/kl/eucld | inv_ws/(kl) |  

- Alt: manche lr/ft_epochs waren immer noch schlecht -> sc_q1_some_datasets_classification_and_selection_get_better_param2 (me5norfz)
- Alt: generelle Tests: fj01kgs5, (new 14cyr3rb)
- Alt  mit opt lr und ft-epochhs lim_domainnet: sr9avnjp

## ---

### Sweep: backbone_and_kd
- Backbone: nbibpxto, KD: pj12t7ux

|               | cifar100 | imagenetr | cub | dil_imagenetr | imageneta | vtab | cars | omnibenchmark | limited_domainnet |
|---------------|----------|-----------|-----|--------------|-----------|------|------|--------------|-------------------|
| Backbone | egal | in21k | in21k | in21k | (in21k) | normal | --- | normal | normal |  
| KD | gleich | gleich | gleich | gleich | gleich | gleich | --- | gleich | <1% |

#### KD:
Bei cifar100: pj12t7ux nicht klar entscheidbar

- Alt: Bessere Parameter herausfinden damit nicht jede acc. 0.1 ist (tp49kob2)
- Alt: Cifar: Bei KD alpha hoch machen, da es bis jetzt immer egal ist: (pj12t7ux). Achtung: wenn kd=False ist kd-alpha egal weil es nicht benutzt wird. Habe ich KD richtig implementiert? Evtl aber auch ungünstige andere parameter gewählt, weil oft acc=random

## ---

### Sweep: lr_ftepoch
  
|        | cifar100 | imagenetr | cub | dil_imagenetr | imageneta | vtab | cars | omnibenchmark | limited_domainnet |
|--------|----------|-----------|-----|--------------|-----------|------|------|--------------|-------------------|
| Selection| bayesian | bayesian | bayesian | bayesian | bayesian | bayesian | --- | bayesian | bayesian |
| Classification  | inv_ws/inv_kl | ws | inv_eucld | ws | ws | kl/eucld | --- | ws/kl/eucld | inv_ws/(kl) |  

- Neu:wie siht der datensatz (domaine oder klassen), wie hsehen die distr. aus. größe des datensatzes und anzahl der images pro klasse. um im vorhinein schon classification auszuwählen. Layup als inspiration. Lr und ft epochs (sollte fest sein) gleich für backbones evtl auch für jeden datensatz (testen mit cifar und dann vtab (klein) und testen ob die lr wirklich so unterschidlich sind)x6p5pa6a und 2ejr8z6d

### Sweep qxb0hd5i: sweep_config_batchsize_optimization
T=10; moe_max_experts=5; reduce_dataset=1
|        | cifar100 | imagenetr | cub | dil_imagenetr | imageneta | vtab | cars | omnibenchmark | limited_domainnet | cddb |
|--------|----------|-----------|-----|--------------|-----------|------|------|--------------|-------------------| ---- |
| GPU Memory| 7851.74 | 9655.29 | 7228.88 | 9655.29 | 6656.36 | 7486.83 | 7186.94 | 9xxx.xx | --- | --- |


## ---

## Kyra Besprechung:
### 25.03.
1. Warum ist das Lernen so ungleich verteilt? Was ist das Kriterium für den Vergleich? Welche beiden Werte werden miteinander verglichen und wie kommen sie zu stande?
2. Acc.=0 ist Logikfehler. Was wird genau predicted und wie? Wie funktioniert der Prozess? Was ist der output und welche experten werden dabei wie benutzt?
3. Wie funktioniert das vergessen und wieviel wird vergessen? Wenn immer nur ein Experte lernt, vergisst er dann wie zu erwarten oder stärkter?
4. Random Selection als Vergleich und als Basis. Ich weis da ja definitiv wie der selection Prozess funktioniert, funktioniert denn alles andere wie zu erwarten?
5. Wenn die Performance random aussieht wird dann immer die gleiche Klasse predicted oder falsche unterschieldiche?  

Generelle Ideen:  
- predict_class_bayes vereinfachen und mich nicht zu doll an die SEED-Implementation halten. Einfach eine Liste mit allen Distr. und keine Aufteilung nach Experten. Würde den dim=3 Tensor vereinfachen. Ich muss für den seection Prozess aber Überblick behalten welcher Expert welche Klasse/Distr. gelernt hat.
- Man kann evtl. nicht einfach die features des ViTs nehmen sondern nur den ersten (cls?) Token nehmen?
- Pseudocode der Funktionen schreiben!! Ist sehr gut für verständniss und späteres beschreiben des Algs.
- Werden die distrs. richtig erstellt? was ist mit der symetrie/gespiegelten features in der distr.? whabe ich das noch drin? Da könnte der Fehler sein.

#### Selection (Warum wird das wissen so ungleich verteilt?):
4. 7s41oxdx angucken und random mit kl_div vergleichen. Wo liegt der fehler? kann ich durch den Vergleich fehlerquellen ausschließen?  
--> kurz angeguckt: kl_div ist besser als random, aber ich weiß nicht genau wann oder wieso
- kl div: https://wandb.ai/belaschindler-university-hamburg/Test%20project/runs/u89ipbvk/overview
- inverted kl div: https://wandb.ai/belaschindler-university-hamburg/Test%20project/runs/bcrcujyc
- evtl unnötig, wenn ich die versuche eh unten abgedeckt habe

#### Classification:





#### klassification aber auch selection (klassification in den Logs):  
  moe_max_expert:  
    values: [5]  
  selection_method:  
    values: ["kl_div", "inv_kl_div"]  
  finetune_epochs:  
    values: [10]  
  lr:  
    values: [0.005]  
  kl_div_test:  
    values: [0, 1, 2]  
    
Sweep: qubquo0z (alt, wahrscheinlich unnütz), a8ap2jey